In [3]:
import pandas as pd
import re
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

# ---------------------- 1. 读取原始数据 ----------------------
raw_file_path = "/Users/syx/Documents/python脚本/大众点评-爬取代码/pet_funeral_shanghai.xlsx"  # 请确认文件路径正确
if not os.path.exists(raw_file_path):
    print(f"❌ 原始文件不存在：{raw_file_path}")
    exit()
df = pd.read_excel(raw_file_path)
print("="*50)
print(f"📊 原始数据概况：总行数={len(df)}，字段={df.columns.tolist()}")
print("="*50)


# ---------------------- 2. 筛选相关商家（保留原逻辑） ----------------------
def filter_pet_funeral_merchants(df):
    core_keywords = ['宠物殡葬', '宠物火化', '宠物善后', '宠物善终','壁葬','安葬','宠物天堂','善终',
                    '宠物安葬', '宠物葬礼', '宠物纪念', '毛孩子殡葬','火化','生命纪念馆']
    keyword_condition = (
        df['name'].astype(str).str.contains('|'.join(core_keywords), case=False) | 
        df['address'].astype(str).str.contains('|'.join(core_keywords), case=False) | 
        df['comment_num'].astype(str).str.contains('|'.join(core_keywords), case=False)
    )
    # 筛选后去重
    df_filtered = df[keyword_condition].drop_duplicates(subset=['name', 'address'], keep='first').reset_index(drop=True)
    return df_filtered

df_related = filter_pet_funeral_merchants(df)


# ---------------------- 3. 核心修复：准确统计“无评分（含0.0分）”的商家 ----------------------
def count_no_star_merchants(df):
    """统计所有无评分的商家（含0.0分、空值、“无评分”文本等）"""
    # 定义“无评分”的判断条件（覆盖所有场景）
    no_star_condition = (
        # 场景1：float格式的0.0分
        (df['star'] == 0.0) | 
        # 场景2：字符串格式的“0.0分”“0分”
        df['star'].astype(str).str.contains(r'^0\.0分$|^0分$', regex=True) | 
        # 场景3：空值、空字符串
        (df['star'].isna()) | (df['star'].astype(str) == '') | 
        # 场景4：明确的“无评分”文本
        (df['star'] == '无评分') | (df['star'] == '暂无评分')
    )
    # 统计符合条件的商家数量
    no_star_count = len(df[no_star_condition])
    # 返回数量和对应的商家数据（便于验证）
    no_star_merchants = df[no_star_condition]
    return no_star_count, no_star_merchants


# ---------------------- 4. 重新统计并验证 ----------------------
# 统计相关商家中的无评分数量
no_star_count, no_star_merchants = count_no_star_merchants(df_related)
# 统计无人均价格的数量（同步优化，避免类似错误）
no_avg_price_count = len(df_related[
    (df_related['avg_price'] == '人均 -') | (df_related['avg_price'].isna()) | (df_related['avg_price'].astype(str) == '')
])
# 统计有完整数据（有人均+有评分）的数量
complete_data_count = len(df_related) - no_avg_price_count - no_star_count


# ---------------------- 5. 保存筛选结果 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
related_file_name = f"pet_funeral_shanghai_cleaned_{timestamp}.xlsx"
related_file_path = os.path.join(os.getcwd(), related_file_name)
df_related.to_excel(related_file_path, index=False, engine='openpyxl')


# ---------------------- 6. 输出准确结果 ----------------------
print("\n" + "="*60)
print(f"🎉 筛选完成！相关商家总行数：{len(df_related)}")
print(f"📁 保存路径：{related_file_path}")
print("\n📋 相关商家数据完整性统计（修复后）：")
print(f"1. 含'人均 -'/无人均的商家：{no_avg_price_count} 家")
print(f"2. 无评分（含0.0分）的商家：{no_star_count} 家")  # 修复后的统计结果
print(f"3. 同时有人均+有评分的商家：{complete_data_count} 家")
print("\n🔍 无评分商家预览（前3家，若有）：")
if no_star_count > 0:
    print(no_star_merchants[['name', 'avg_price', 'star', 'address']].head(3))
else:
    print("   暂无无评分的相关商家")
print("="*60)


# 可选：验证原始评分字段的格式（帮助你确认数据实际情况）
print(f"\n📊 原始评分字段（star）的唯一值示例：")
unique_stars = df_related['star'].unique()[:10]  # 打印前10个唯一值
for star in unique_stars:
    print(f"   格式={type(star).__name__}, 值='{star}'")

📊 原始数据概况：总行数=315，字段=['page', 'index', 'name', 'link', 'address', 'avg_price', 'comment_num', 'star']

🎉 筛选完成！相关商家总行数：126
📁 保存路径：/Users/syx/Documents/python脚本/pet_funeral_shanghai_cleaned_20251113_140525.xlsx

📋 相关商家数据完整性统计（修复后）：
1. 含'人均 -'/无人均的商家：25 家
2. 无评分（含0.0分）的商家：41 家
3. 同时有人均+有评分的商家：60 家

🔍 无评分商家预览（前3家，若有）：
                   name avg_price  star address
0            宠物殡葬VIP终点站   人均 ￥865  0.0分     美兰湖
5  屿念·宠物火化殡葬服务机构·24小时服务      人均 -  0.0分     虹梅路
7   星辰·宠物火化善终服务机构(灵山路店)      人均 -  0.0分      洋泾

📊 原始评分字段（star）的唯一值示例：
   格式=str, 值='0.0分'
   格式=str, 值='4.0分'
   格式=str, 值='5.0分'
   格式=str, 值='4.5分'
   格式=str, 值='3.5分'
   格式=str, 值='3.0分'


In [4]:
import pandas as pd
import os
from datetime import datetime

# ---------------------- 配置参数（请修改为你的实际路径） ----------------------
INPUT_RAW_DATA = "/Users/syx/Documents/python脚本/pet_funeral_shanghai_cleaned_20251113_140525.xlsx"  # 上海原始数据路径
OUTPUT_CLEANED_DATA = f"shanghai_second_cleaned_{datetime.now().strftime('%Y%m%d')}.xlsx"  # 输出路径


# ---------------------- 核心：上海地址精准分区函数 ----------------------
def address_to_district(address):
    """将地址映射到上海行政区（覆盖美兰湖、金桥、康桥等所有提供的地点）"""
    if pd.isna(address):
        return "地址缺失"
    
    addr = str(address).strip().replace(" ", "").lower()
    if addr == "":
        return "地址缺失"

    # 上海地点-行政区映射表（严格按上海最新行政规划，覆盖所有提供的地点）
    district_mapping = {
        # 浦东新区（含金桥、洋泾、康桥等）
        "浦东新区": ["浦东新区", "浦东", "金桥", "洋泾", "康桥", "周浦", "航头", 
                  "三林地区", "塘桥", "外高桥", "陆家嘴", "金杨地区", "临沂", "南码头",
                  "上南地区", "董家渡", "南浦大桥", "临平路", "和平公园"],
        # 闵行区（含浦江镇、颛桥等）
        "闵行区": ["闵行区", "闵行", "浦江镇", "颛桥", "北桥", "曹行镇", "春申地区", 
                 "莲花路", "南方商城", "七宝", "虹桥镇", "莘庄"],
        # 宝山区（含美兰湖、顾村公园等）
        "宝山区": ["宝山区", "宝山", "美兰湖", "顾村公园", "宝山城区", "吴淞", "大华地区",
                 "通河", "泗塘", "上海大学", "共富新村", "高境", "庙行", "共康"],
        # 嘉定区（含嘉定镇、南翔等）
        "嘉定区": ["嘉定区", "嘉定", "嘉定镇", "南翔", "嘉定新城", "江桥", "丰庄", 
                 "江桥万达广场"],
        # 杨浦区（含新江湾城、五角场等）
        "杨浦区": ["杨浦区", "杨浦", "新江湾城", "黄兴公园", "平凉路", "东外滩", 
                 "五角场", "大学路"],
        # 静安区（含南京西路、静安寺等）
        "静安区": ["静安区", "静安", "南京西路", "上海火车站", "客运汽车总站", "海宁路", 
                 "七浦路", "曹家渡", "彭浦镇", "彭浦新村", "大宁地区", "苏河湾", 
                 "同乐坊", "江宁路", "市北工业园", "汶水路", "闸北公园", "静安寺"],
        # 黄浦区（含淮海路、城隍庙等）
        "黄浦区": ["黄浦区", "黄浦", "淮海路", "城隍庙", "豫园", "人民广场", "南京路", 
                 "新天地", "马当路", "肇嘉浜路", "中山医院"],
        # 徐汇区（含虹梅路、龙华等）
        "徐汇区": ["徐汇区", "徐汇", "虹梅路", "龙华", "西岸", "上海南站"],
        # 普陀区（含真如、曹杨等）
        "普陀区": ["普陀区", "普陀", "真如", "中山北路", "甘泉地区", "长风公园", "华师大", 
                 "曹杨地区", "西站大华", "桃浦", "长寿路"],
        # 长宁区（含中山公园、天山等）
        "长宁区": ["长宁区", "长宁", "中山公园", "江苏路", "天山"],
        # 虹口区（含四川北路、凉城等）
        "虹口区": ["虹口区", "虹口", "四川北路", "海伦路", "凉城", "江湾镇"],
        # 松江区（含松江万达、佘山等）
        "松江区": ["松江区", "松江", "松江万达广场", "松江大学城", "佘山", "洞泾", "泗泾镇"],
        # 金山区（含金山新城、朱泾镇等）
        "金山区": ["金山区", "金山", "金山新城", "朱泾镇"],
        # 青浦区（含青浦城区等）
        "青浦区": ["青浦区", "青浦", "青浦城区"],
        # 奉贤区（含海湾旅游区等）
        "奉贤区": ["奉贤区", "奉贤", "海湾旅游区"]
    }

    # 精准匹配：优先匹配完整关键词（如“康桥/周浦”会匹配“康桥”或“周浦”）
    for district, keywords in district_mapping.items():
        for kw in keywords:
            if kw in addr:
                return district

    return "待确认（需人工核对）"  # 未匹配到的地址（极少）


# ---------------------- 执行清洗分区 ----------------------
if not os.path.exists(INPUT_RAW_DATA):
    print(f"❌ 错误：未找到原始数据文件，请检查路径：{INPUT_RAW_DATA}")
else:
    # 读取原始数据
    df = pd.read_excel(INPUT_RAW_DATA)
    if "address" not in df.columns:
        print(f"❌ 错误：原始数据中未找到 'address' 列，请确认列名正确")
    else:
        # 处理address列，新增行政区列（district_cn）
        df["district_cn"] = df["address"].apply(address_to_district)
        
        # 统计分区结果（方便检查准确性）
        print(f"\n📊 地址分区结果统计（共{len(df)}条数据）：")
        district_stats = df["district_cn"].value_counts()
        for dist, cnt in district_stats.items():
            print(f"   {dist}：{cnt}条（占比：{cnt/len(df)*100:.1f}%）")
        
        # 保存清洗后的数据（含行政区列）
        df.to_excel(OUTPUT_CLEANED_DATA, index=False, engine="openpyxl")
        print(f"\n✅ 清洗完成！带行政区的数据已保存至：{OUTPUT_CLEANED_DATA}")
        print(f"   （请先检查该文件中的 'district_cn' 列，确认分区正确后，再运行热力图代码）")


📊 地址分区结果统计（共124条数据）：
   浦东新区：24条（占比：19.4%）
   宝山区：16条（占比：12.9%）
   闵行区：14条（占比：11.3%）
   静安区：14条（占比：11.3%）
   普陀区：10条（占比：8.1%）
   嘉定区：10条（占比：8.1%）
   杨浦区：8条（占比：6.5%）
   松江区：6条（占比：4.8%）
   黄浦区：5条（占比：4.0%）
   金山区：4条（占比：3.2%）
   徐汇区：4条（占比：3.2%）
   长宁区：3条（占比：2.4%）
   青浦区：3条（占比：2.4%）
   虹口区：2条（占比：1.6%）
   奉贤区：1条（占比：0.8%）

✅ 清洗完成！带行政区的数据已保存至：shanghai_second_cleaned_20251113.xlsx
   （请先检查该文件中的 'district_cn' 列，确认分区正确后，再运行热力图代码）
